In [2]:
from selenium.webdriver import Chrome
from utils.login import login_glassdoor
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [14]:
driver = Chrome()
driver2 = Chrome()
driver.get("https://www.glassdoor.fr/Avis/index.htm")
login_glassdoor(driver, "severin.lefebure@edu.devinci.fr", "jesuisungenie")

<selenium.webdriver.chrome.webdriver.WebDriver (session="d637f8675b98843609960a832ccaf59e")>

In [15]:
login_glassdoor(driver2, "severin.lefebure@edu.devinci.fr", "jesuisungenie")

<selenium.webdriver.chrome.webdriver.WebDriver (session="1bf1387752932bec735923f541e375ea")>

In [16]:
driver.implicitly_wait(0.01)
driver2.implicitly_wait(0.01)

In [18]:
driver.get("https://www.glassdoor.fr/Avis/index.htm")

employers = driver.find_elements(By.CSS_SELECTOR, '[data-test="employer-card-single"]')
employer_data = {}

for employer in employers:
    name = employer.find_element(By.CSS_SELECTOR, '[data-test="employer-short-name"]').text
    print(name)
    ranking = employer.find_element(By.CSS_SELECTOR, '[data-test="rating"]').text
    print(ranking)
    employer_data[name] = {"Rank": ranking, "Reviews": []}

    
for employer in employers:
    name = employer.find_element(By.CSS_SELECTOR, '[data-test="employer-short-name"]').text
    print(name)
    link = employer.find_element(By.CSS_SELECTOR, '[data-test="cell-Reviews-url"]').get_attribute("href")  
    print(link)
    driver2.get(link)
    # Reviews
    reviews = driver2.find_elements(By.CLASS_NAME, "review-details__review-details-module__reviewDetails")[:20]
    
    reviews_data = []
    
    for review in reviews:
        hover = ActionChains(driver2).move_to_element(review).click()
        hover.perform()
        rating = review.find_element(By.CLASS_NAME, "review-details__review-details-module__overallRating").text
        date = review.find_element(By.CLASS_NAME, "review-details__review-details-module__reviewDate").text
        job_type = review.find_element(By.CLASS_NAME, "review-details__review-details-module__employee").text
        details = review.find_elements(By.CLASS_NAME, "mr-std.review-details__review-details-module__ratingDetail")
        details_list = []
        for detail in details:
            title = detail.find_element(By.CLASS_NAME, "review-details__review-details-module__ratingTitle").text
            try:
                detail.find_element(By.CLASS_NAME, "review-details__review-details-module__negativeStyles")
                sentiment = "bad"
            except:
                try:
                    detail.find_element(By.CLASS_NAME, "review-details__review-details-module__positiveStyles")
                    sentiment = "good"
                except:
                    try:
                        detail.find_element(By.CLASS_NAME, "review-details__review-details-module__neutralStyles")
                        sentiment = "neutral"
                    except:
                        detail.find_element(By.CLASS_NAME, "review-details__review-details-module__noDataStyles")
                        sentiment = "None"
            details_list.append((title,sentiment))
        
        pros = review.find_element(By.CSS_SELECTOR, '[data-test="pros"]').text
        cons = review.find_element(By.CSS_SELECTOR, '[data-test="cons"]').text 
        
        print((rating,date,job_type,details_list,pros,cons))
        
        reviews_data.append((rating,date,job_type,details_list,pros,cons))
    
    employer_data[name]["reviews"] = reviews_data
    
    with open('dict_reviews.txt','w', encoding="utf-8") as f:
        f.write(employer_data.__str__())


McDonald's
3,5
Amazon
3,7
Accenture
3,8
Deloitte
4,0
Capgemini
3,6
EY
3,8
PwC
3,9
KPMG
3,8
IBM
4,1
BNP Paribas
3,8
McDonald's
https://www.glassdoor.fr/Avis/McDonald-s-Avis-E432.htm
('5,0', '14 janv. 2024', 'Assistant Manager (McDonalds)', [('Recommander', 'None'), ('Approbation du PDG', 'None'), ('Perspective commerciale', 'None')], 'Bien payer et très bien organisé', 'Les horaires parfois sont moins bien')
('2,0', '11 déc. 2023', 'Employé anonyme', [('Recommander', 'None'), ('Approbation du PDG', 'None'), ('Perspective commerciale', 'None')], "Flexibilité des horaires : McDonald's offre souvent des horaires flexibles, ce qui peut convenir aux étudiants, aux travailleurs à temps partiel ou à ceux qui ont des engagements personnels. Opportunités d'avancement : McDonald's encourage souvent la promotion interne, offrant aux employés la possibilité d'avancer dans leur carrière au sein de l'entreprise. Expérience professionnelle : Travailler au McDonald's peut constituer une première expéri

In [19]:
dict_reviews = eval(open('dict_reviews.txt', encoding="utf-8").read())

In [20]:
dict_reviews.keys()

dict_keys(["McDonald's", 'Amazon', 'Accenture', 'Deloitte', 'Capgemini', 'EY', 'PwC', 'KPMG', 'IBM', 'BNP Paribas'])

In [22]:
import pandas as pd
df = []
names = ["Company", "Rank", "Review_score", "Date_review", "Job", "Recommandation", "Approb CEO", "Perspective", "Pros", "Cons"]
for key, value in dict_reviews.items():
    print(key, value)
    rank = value["Rank"]
    for review in value["reviews"]:
        print(review[-1])
        row = review[:-1] + [tag[1] for tag in review[-1]]
        df.append(row)

df=pd.DataFrame(df,columns=names)
df
        

McDonald's {'Rank': '3,5', 'Reviews': [], 'reviews': [('5,0', '14 janv. 2024', 'Assistant Manager (McDonalds)', [('Recommander', 'None'), ('Approbation du PDG', 'None'), ('Perspective commerciale', 'None')], 'Bien payer et très bien organisé', 'Les horaires parfois sont moins bien'), ('2,0', '11 déc. 2023', 'Employé anonyme', [('Recommander', 'None'), ('Approbation du PDG', 'None'), ('Perspective commerciale', 'None')], "Flexibilité des horaires : McDonald's offre souvent des horaires flexibles, ce qui peut convenir aux étudiants, aux travailleurs à temps partiel ou à ceux qui ont des engagements personnels. Opportunités d'avancement : McDonald's encourage souvent la promotion interne, offrant aux employés la possibilité d'avancer dans leur carrière au sein de l'entreprise. Expérience professionnelle : Travailler au McDonald's peut constituer une première expérience professionnelle pour de nombreux jeunes, offrant des compétences de base en service à la clientèle, en gestion du temps e

IndexError: string index out of range